# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [39]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config2 import api_key

gmaps.configure(api_key=api_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [60]:
weather_data = pd.read_csv("../WeatherPy/Weather_Data_Export.csv")
weather_data

,city,lat,lng,country,temp,humidity,clouds,wind
0,Iqaluit,63.75,-68.51,CA,5.00,77,75,2.82
1,Taylorville,39.55,-89.29,US,52.36,37,1,6.93
2,Linares,-35.85,-71.60,CL,95.67,8,0,12.08
3,Rikitea,-23.12,-134.97,PF,76.96,76,74,20.18
4,Green River,41.53,-109.47,US,42.80,56,1,28.86
...,...,...,...,...,...,...,...,...
564,Kodinsk,58.69,99.18,RU,14.34,96,78,7.34
565,Mezen',65.85,44.24,RU,10.45,92,0,4.32
566,Lompoc,34.64,-120.46,US,63.97,82,20,5.82
567,Taksimo,56.34,114.88,RU,-7.62,95,37,1.66


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [42]:
locations = weather_data[["lat", "lng"]]
weight = weather_data["humidity"].astype(float)

In [43]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight,
                                dissipating=False, max_intensity=100,
                                point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [50]:
hot_weather = weather_data.loc[weather_data["temp"] >= 72, :]
perfect_temp = hot_weather.loc[hot_weather["temp"] <= 80, :]
perfect_wind = perfect_temp.loc[perfect_temp["wind"] <= 10, :]
perfect_wind = perfect_wind.loc[perfect_wind["wind"] > 2, :]
perfect_clouds = perfect_wind.loc[perfect_wind["clouds"] == 0, :]
hotel_df = perfect_clouds.loc[perfect_clouds["humidity"] < 80, :]
hotel_df = hotel_df.loc[hotel_df["humidity"] > 5, :]
hotel_df

,city,lat,lng,country,temp,humidity,clouds,wind
179,Abū Zabad,12.35,29.25,SD,76.08,10,0,9.60
261,Qandala,11.47,49.87,SO,74.53,77,0,3.40
331,Buta,2.79,24.73,CD,74.88,68,0,2.80
346,Aketi,2.74,23.78,CD,77.52,60,0,3.29
361,Bilma,18.69,12.92,NE,73.99,8,0,6.44
451,Necochea,-38.55,-58.74,AR,78.21,66,0,3.00
468,Abéché,13.83,20.83,TD,77.50,17,0,6.29
495,Atar,20.52,-13.05,MR,78.40,7,0,7.09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [51]:
hotel_df["Hotel Name"] = ""
hotel_df

,city,lat,lng,country,temp,humidity,clouds,wind,Hotel Name
179,Abū Zabad,12.35,29.25,SD,76.08,10,0,9.60,
261,Qandala,11.47,49.87,SO,74.53,77,0,3.40,
331,Buta,2.79,24.73,CD,74.88,68,0,2.80,
346,Aketi,2.74,23.78,CD,77.52,60,0,3.29,
361,Bilma,18.69,12.92,NE,73.99,8,0,6.44,
451,Necochea,-38.55,-58.74,AR,78.21,66,0,3.00,
468,Abéché,13.83,20.83,TD,77.50,17,0,6.29,
495,Atar,20.52,-13.05,MR,78.40,7,0,7.09,


In [53]:
query_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

target_search = "Hotel"
target_radius = 5000
target_type = "hotel"

params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": api_key
}

for index, row in hotel_df.iterrows():
    try:
        city_name = row["city"]
        lat = row["lat"]
        lng = row["lng"]
        params["location"] = f"{lat},{lng}"
        print(f"Retrieving results for {city_name}")
        response = requests.get(query_url, params=params).json()
        results = response["results"]
        print(results[0]['name'])
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except:
        print("Missing some data...skipping")

hotel_df = hotel_df.dropna()
hotel_df

Retrieving results for Abū Zabad
Missing some data...skipping
Retrieving results for Qandala
Missing some data...skipping
Retrieving results for Buta
HOTEL BISO NA BISO
Retrieving results for Aketi
Missing some data...skipping
Retrieving results for Bilma
Missing some data...skipping
Retrieving results for Necochea
Hotel Real
Retrieving results for Abéché
Hotel Cesar
Retrieving results for Atar
Auberge mer et desert


,city,lat,lng,country,temp,humidity,clouds,wind,Hotel Name
179,Abū Zabad,12.35,29.25,SD,76.08,10,0,9.60,
261,Qandala,11.47,49.87,SO,74.53,77,0,3.40,
331,Buta,2.79,24.73,CD,74.88,68,0,2.80,HOTEL BISO NA BISO
346,Aketi,2.74,23.78,CD,77.52,60,0,3.29,
361,Bilma,18.69,12.92,NE,73.99,8,0,6.44,
451,Necochea,-38.55,-58.74,AR,78.21,66,0,3.00,Hotel Real
468,Abéché,13.83,20.83,TD,77.50,17,0,6.29,Hotel Cesar
495,Atar,20.52,-13.05,MR,78.40,7,0,7.09,Auberge mer et desert


In [54]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>Country</dt><dd>{country}</dd>
<dt>City</dt><dd>{city}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_2 = hotel_df[["lat", "lng"]]

In [55]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations_2, info_box_content=hotel_info)
fig.add_layer(markers)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))